In [1]:
from keras.models import Model, Sequential
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from PIL import Image
import numpy as np
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
import matplotlib.pyplot as plt


def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img


def findCosineSimilarity(source_representation, test_representation):
    a = np.matmul(np.transpose(source_representation), test_representation)
    b = np.sum(np.multiply(source_representation, source_representation))
    c = np.sum(np.multiply(test_representation, test_representation))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))


def findEuclideanDistance(source_representation, test_representation):
    euclidean_distance = source_representation - test_representation
    euclidean_distance = np.sum(np.multiply(euclidean_distance, euclidean_distance))
    euclidean_distance = np.sqrt(euclidean_distance)
    return euclidean_distance

Using TensorFlow backend.


In [2]:
print("Constructing model")
model = Sequential()
model.add(ZeroPadding2D((1, 1), input_shape=(224, 224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))
# you can download the pretrained weights from the following link
# https://drive.google.com/file/d/1CPSeum3HpopfomUEK1gybeuIVoeJT_Eo/view?usp=sharing
# or you can find the detailed documentation https://sefiks.com/2018/08/06/deep-face-recognition-with-keras/

from keras.models import model_from_json
print("loading weights")
model.load_weights('vgg_face_weights.h5')
print("weights loaded")

Constructing model


loading weights


weights loaded


In [3]:
model.pop()
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='softmax'))

#set only last layer to be trainable
for layer in model.layers:
    layer.trainable = False
model.layers[-1].trainable = True
model.layers[-2].trainable = True
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_1 (ZeroPaddin (None, 226, 226, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 226, 226, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 114, 114, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     73856     
__________

In [ ]:
# model.summary()
vgg_face_descriptor = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output) # Functional model
vgg_face_descriptor.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_1_input (Inpu (None, 224, 224, 3)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 226, 226, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 226, 226, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 114, 114, 64)      0         
__________

In [5]:
import os

def getTrainingData():
    train_imgs = []
    labels = []
    for file in os.listdir("datasets/ReplayAttackIMGs/attack_hand"):
        train_imgs.append(preprocess_image("datasets/ReplayAttackIMGs/attack_hand/" + file))
        labels.append(0)
    for file in os.listdir("datasets/ReplayAttackIMGs/real"):
        train_imgs.append(preprocess_image("datasets/ReplayAttackIMGs/real/" + file))
        labels.append(1)
    train_imgs = np.array(train_imgs)
    train_imgs = train_imgs[:, 0, :, :]
    labels = np.array(labels)
    return train_imgs, labels

tr_imgs, tr_labels = getTrainingData()
print(tr_imgs.shape, tr_labels.shape)



(2400, 224, 224, 3) (2400,)


In [35]:
def getTestData():
    test_imgs = []
    labels = []
    for file in os.listdir("datasets/ReplayAttackIMGs_test/attack_hand"):
        test_imgs.append(preprocess_image("datasets/ReplayAttackIMGs_test/attack_hand/" + file))
        labels.append(0)
    for file in os.listdir("datasets/ReplayAttackIMGs_test/real"):
        test_imgs.append(preprocess_image("datasets/ReplayAttackIMGs_test/real/" + file))
        labels.append(1)
    test_imgs = np.array(test_imgs)
    test_imgs = test_imgs[:, 0, :, :]
    labels = np.array(labels)
    return test_imgs, labels

test_imgs, test_labels = getTestData()
print(test_imgs.shape, test_labels.shape)


(3200, 224, 224, 3) (3200,)


In [12]:
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
from keras.metrics import binary_accuracy

optimizer = Adam()
model.compile(optimizer='sgd', loss=binary_crossentropy, metrics=['accuracy',binary_accuracy])

In [14]:
history = model.fit(tr_imgs, tr_labels, batch_size=1, epochs=2)
print(history.history)

Epoch 1/2


   1/2400 [..............................] - ETA: 9:11 - loss: 15.9424 - acc: 0.0000e+00 - binary_accuracy: 0.0000e+00

   2/2400 [..............................] - ETA: 9:00 - loss: 7.9712 - acc: 0.5000 - binary_accuracy: 0.5000         

   3/2400 [..............................] - ETA: 9:00 - loss: 10.6283 - acc: 0.3333 - binary_accuracy: 0.3333

   4/2400 [..............................] - ETA: 8:56 - loss: 7.9712 - acc: 0.5000 - binary_accuracy: 0.5000 

...

2397/2400 [============================>.] - ETA: 0s - loss: 9.9565 - acc: 0.3755 - binary_accuracy: 0.3755

2398/2400 [============================>.] - ETA: 0s - loss: 9.9590 - acc: 0.3753 - binary_accuracy: 0.3753

2399/2400 [============================>.] - ETA: 0s - loss: 9.9615 - acc: 0.3752 - binary_accuracy: 0.3752

2400/2400 [==============================] - 527s 219ms/step - loss: 9.9640 - acc: 0.3750 - binary_accuracy: 0.3750


{'loss': [9.963990494608879, 9.963990494608879], 'acc': [0.375, 0.375], 'binary_accuracy': [0.375, 0.375]}


In [13]:
model.save("MyVGGface.h5")

In [14]:
from keras.models import load_model
model = load_model("MyVGGface.h5")

In [19]:
score = model.evaluate(test_imgs,test_labels,batch_size=1 )
print(score)

   1/3200 [..............................] - ETA: 12:30

   2/3200 [..............................] - ETA: 11:53

   3/3200 [..............................] - ETA: 11:59

   4/3200 [..............................] - ETA: 11:55

   5/3200 [..............................] - ETA: 12:00

...

3197/3200 [============================>.] - ETA: 0s

3198/3200 [============================>.] - ETA: 0s

3199/3200 [============================>.] - ETA: 0s

3200/3200 [==============================] - 680s 213ms/step


19.927980944514275


In [ ]:
scores = model.predict(test_imgs,batch_size=1,verbose=1)